In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# ----------------------------
# 1. Generate Synthetic Data
# ----------------------------

def generate_forest_image(forest_type):
    base = np.random.rand(64, 64, 4)
    if forest_type == 0:  # Tropical
        base[:, :, 3] += 0.2  # Higher NIR
        base[:, :, 1] += 0.1  # Lush Green
    elif forest_type == 1:  # Temperate
        base[:, :, 0] += 0.1  # Slightly more red
        base[:, :, 2] += 0.1  # More balanced
    elif forest_type == 2:  # Boreal
        base[:, :, 0] += 0.05
        base[:, :, 3] -= 0.1  # Less NIR (sparse vegetation)
    return np.clip(base, 0, 1)

n_samples = 1500
X = np.zeros((n_samples, 64, 64, 4))
y = np.zeros(n_samples)

for i in range(n_samples):
    label = i // 500  # 3 classes: 0, 1, 2
    X[i] = generate_forest_image(label)
    y[i] = label

# ----------------------------
# 2. Train-Test Split
# ----------------------------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_train_cat = tf.keras.utils.to_categorical(y_train, 3)
y_test_cat = tf.keras.utils.to_categorical(y_test, 3)

# ----------------------------
# 3. CNN Model
# ----------------------------

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 4)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------
# 4. Train Model
# ----------------------------

history = model.fit(X_train, y_train_cat, epochs=10, batch_size=32, validation_split=0.2)

# ----------------------------
# 5. Evaluate Model
# ----------------------------

loss, acc = model.evaluate(X_test, y_test_cat)
print(f"\nTest Accuracy: {acc:.2f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Tropical', 'Temperate', 'Boreal']))

# ----------------------------
# 6. Plot Results
# ----------------------------

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
